In [6]:
!pip install spacy_transformers
! pip install -U spacy


In [7]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [8]:
cv_data = json.load(open('CV-Parsing-using-Spacy-3/data/training/train_data.json'))

In [9]:
cv_data[0]

['Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [10]:
len(cv_data)

200

In [20]:
!python -m spacy init fill-config base_configtransf.cfg config.cfg

/Users/choukigueddar/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [21]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.3)

In [22]:
len(train), len(test)

(140, 60)

In [23]:
type(train)

list

In [24]:
def get_spacy_doc(file, data):
    nlp = spacy.blank('en')
    db = DocBin()

    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)

        # Vérifiez si annot est un dictionnaire et s'il contient la clé 'entities'
        if isinstance(annot, dict) and 'entities' in annot:
            annot_entities = annot['entities']
        else:
            # Gérez le cas où annot n'est pas conforme à vos attentes
            print(f"Warning: Invalid 'annot' format for text: {text}")
            continue

        ents = []
        entity_indices = []

        for start, end, label in annot_entities:
            skip_entity = False
            for idx in range(start, end):
                if idx in entity_indices:
                    skip_entity = True
                    break
            if skip_entity:
                continue

            entity_indices += list(range(start, end))

            try:
                span = doc.char_span(start, end, label=label, alignment_mode='strict')
            except:
                continue

            if span is None:
                err_data = str([start, end]) + "     " + str(text) + "\n"
                file.write(err_data)
            else:
                ents.append(span)

        try:
            doc.ents = ents
            db.add(doc)
        except:
            pass

    return db

In [25]:
file = open('error.txt', 'w')

db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

file.close()

100%|██████████████████████████████████████████| 60/60 [00:00<00:00, 192.44it/s]


In [26]:
len(db.tokens)

60

In [27]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 5.0 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [28]:
!python -m spacy train config.cfg --output ./output --paths.train train_data.spacy --paths.dev test_data.spacy

ℹ Saving to output directory: output
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
/Users/choukigueddar/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'ner']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  -------------  -

In [29]:
nlp = spacy.load('output/model-best')


In [30]:
def extract_text_from_pdf(f):
    text = ""
    #with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    num_pages = len(reader.pages)
    for page_num in range(num_pages):
        page = reader.pages[page_num]
        text += page.extract_text()
    return text


In [32]:
!pip install PyMuPDF

In [33]:
pip uninstall fitz==0.0.1.dev2

Note: you may need to restart the kernel to use updated packages.


In [20]:
text = " "
for page in doc:
  text = text + str(page.get_text())

NameError: name 'doc' is not defined

In [21]:
text = text.strip()

In [22]:
' '.join(text.split())

''

In [23]:
doc = nlp(text)
for ent in doc.ents:
  print(ent.label_,":", ent.text)